In [ ]:
!pip install albumentations==0.4.6
import os
import numpy as np
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
from torch import nn,optim
import torch
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import matplotlib.pyplot as plt


class UNET(nn.Module):
            def __init__(self):
                        super().__init__()
                        
                        #DOWN SAMPLE LAYERS

                        self.conv1_down_1 = nn.Conv2d(3,64,3,padding=1)
                        self.bn1_down_1 = nn.BatchNorm2d(64)
                        self.conv1_down_2 = nn.Conv2d(64,64,3,padding=1)
                        self.bn1_down_2 = nn.BatchNorm2d(64)

                        self.conv2_down_1 = nn.Conv2d(64,128,3,padding=1)
                        self.bn2_down_1 = nn.BatchNorm2d(128)
                        self.conv2_down_2 = nn.Conv2d(128,128,3,padding=1)
                        self.bn2_down_2 = nn.BatchNorm2d(128)
                        
                        self.conv3_down_1 = nn.Conv2d(128,256,3,padding=1)
                        self.bn3_down_1 = nn.BatchNorm2d(256)
                        self.conv3_down_2 = nn.Conv2d(256,256,3,padding=1)
                        self.bn3_down_2 = nn.BatchNorm2d(256)

                        self.conv4_down_1 = nn.Conv2d(256,512,3,padding=1)
                        self.bn4_down_1 = nn.BatchNorm2d(512)
                        self.conv4_down_2 = nn.Conv2d(512,512,3,padding=1)
                        self.bn4_down_2 = nn.BatchNorm2d(512)

                        self.conv5_down_1 = nn.Conv2d(512,1024,3,padding=1)
                        self.bn5_down_1 = nn.BatchNorm2d(1024)
                        self.conv5_down_2 = nn.Conv2d(1024,1024,3,padding=1)
                        self.bn5_down_2 = nn.BatchNorm2d(1024)

                        self.pool = nn.MaxPool2d(2,2)

                        #UP SAMPLE LAYERS

                        self.up_sample_1 = nn.ConvTranspose2d(1024,512,2,2)
                        self.conv1_up_1 = nn.Conv2d(1024,512,3,padding=1)
                        self.bn1_up_1 = nn.BatchNorm2d(512)
                        self.conv1_up_2 = nn.Conv2d(512,512,3,padding=1)
                        self.bn1_up_2 = nn.BatchNorm2d(512)

                        self.up_sample_2 = nn.ConvTranspose2d(512,256,2,2)
                        self.conv2_up_1 = nn.Conv2d(512,256,3,padding=1)
                        self.bn2_up_1 = nn.BatchNorm2d(256)
                        self.conv2_up_2 = nn.Conv2d(256,256,3,padding=1)
                        self.bn2_up_2 = nn.BatchNorm2d(256)

                        self.up_sample_3 = nn.ConvTranspose2d(256,128,2,2)
                        self.conv3_up_1 = nn.Conv2d(256,128,3,padding=1)
                        self.bn3_up_1 = nn.BatchNorm2d(128)
                        self.conv3_up_2 = nn.Conv2d(128,128,3,padding=1)
                        self.bn3_up_2 = nn.BatchNorm2d(128)

                        self.up_sample_4 = nn.ConvTranspose2d(128,64,2,2)
                        self.conv4_up_1 = nn.Conv2d(128,64,3,padding=1)
                        self.bn4_up_1 = nn.BatchNorm2d(64)
                        self.conv4_up_2 = nn.Conv2d(64,64,3,padding=1)
                        self.bn4_up_2 = nn.BatchNorm2d(64)

                        self.output = nn.Conv2d(64,1,1)

            def forward(self,x):

                        #DOWN SAMPLING
                        print(x.shape)
                        cd1 = self.conv1_down_1(x)
                        cd1 = self.bn1_down_1(cd1)
                        cd1 = F.relu(cd1)
                        cd1 = self.conv1_down_2(cd1)
                        cd1 = self.bn1_down_2(cd1)
                        cd1 = F.relu(cd1)
                        
                        cd2 = self.pool(cd1)
                        
                        cd2 = self.conv2_down_1(cd2)
                        cd2 = self.bn2_down_1(cd2)
                        cd2 = F.relu(cd2)
                        cd2 = self.conv2_down_2(cd2)
                        cd2 = self.bn2_down_2(cd2)
                        cd2 = F.relu(cd2)
                        
                        cd3 = self.pool(cd2)
                        
                        cd3 = self.conv3_down_1(cd3)
                        cd3 = self.bn3_down_1(cd3)
                        cd3 = F.relu(cd3)
                        cd3 = self.conv3_down_2(cd3)
                        cd3 = self.bn3_down_2(cd3)
                        cd3 = F.relu(cd3)
                        
                        cd4 = self.pool(cd3)
                        
                        cd4 = self.conv4_down_1(cd4)
                        cd4 = self.bn4_down_1(cd4)
                        cd4 = F.relu(cd4)
                        cd4 = self.conv4_down_2(cd4)
                        cd4 = self.bn4_down_2(cd4)
                        cd4 = F.relu(cd4)
                        
                        cd5 = self.pool(cd4)
                        
                        cd5 = self.conv5_down_1(cd5)
                        cd5 = self.bn5_down_1(cd5)
                        cd5 = F.relu(cd5)
                        cd5 = self.conv5_down_2(cd5)
                        cd5 = self.bn5_down_2(cd5)
                        cd5 = F.relu(cd5)
                        
                        #UP SAMPLING

                        cu = self.up_sample_1(cd5)
                        cu = torch.cat((cd4,cu), dim=1)

                        cu = self.conv1_up_1(cu)
                        cu = self.bn1_up_1(cu)
                        cu = F.relu(cu)
                        cu = self.conv1_up_2(cu)
                        cu = self.bn1_up_2(cu)
                        cu = F.relu(cu)

                        cu = self.up_sample_2(cu)
                        cu = torch.cat((cd3,cu), dim=1)

                        cu = self.conv2_up_1(cu)
                        cu = self.bn2_up_1(cu)
                        cu = F.relu(cu)
                        cu = self.conv2_up_2(cu)
                        cu = self.bn2_up_2(cu)
                        cu = F.relu(cu)

                        cu = self.up_sample_3(cu)
                        cu = torch.cat((cd2,cu), dim=1)
                        

                        cu = self.conv3_up_1(cu)
                        cu = self.bn3_up_1(cu)
                        cu = F.relu(cu)
                        cu = self.conv3_up_2(cu)
                        cu = self.bn3_up_2(cu)
                        cu = F.relu(cu)

                        cu = self.up_sample_4(cu)
                        cu = torch.cat((cd1,cu), dim=1)

                        cu = self.conv4_up_1(cu)
                        cu = self.bn4_up_1(cu)
                        cu = F.relu(cu)
                        cu = self.conv4_up_2(cu)
                        cu = self.bn4_up_2(cu)
                        cu = F.relu(cu)

                        output = self.output(cu)

                        return output


class DATASET(Dataset):
            def __init__(self,image_dir,mask_dir,transform=None):
                        self.image_dir = image_dir
                        self.mask_dir = mask_dir
                        self.transform = transform
                        self.images = os.listdir(image_dir)

            def __len__(self):
                        return len(self.images)

            def __getitem__(self,index):
                        image_path = os.path.join(self.image_dir,self.images[index])
                        mask_path = os.path.join(self.mask_dir,"seg"+self.images[index])
                        image = cv2.imread(image_path)
                        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                        mask = cv2.imread(mask_path,0)
                        ret,mask=cv2.threshold(mask,127,255,cv2.THRESH_BINARY)
                        mask=mask.astype(np.float32)/255.0
                        if not self.transform == None:
                                    augmentations = self.transform(image= image,mask=mask)
                                    image = augmentations["image"]
                                    mask = augmentations["mask"]

                        return image,mask

def plotVals(
    losses,
    ylim=(0, 1.5),
    f_name="Losses",
    x_label="Training Step",
    y_label="Training Loss",
):
    x_axis = np.arange(len(losses))
    plt.plot(x_axis, losses, label=y_label + " Curve")
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.ylim(ylim)
    plt.legend()
    plt.title(y_label + " Curve")
    plt.savefig("/content/drive/MyDrive/Segmentation/MRI SEG/Plots/Binary/{}/".format(f_name) + y_label + ".png", bbox_inches="tight")
    plt.close()


training_losses = []
validation_losses = []
dice_scores = []

train_transform = A.Compose([A.Resize(height=256,width=256),
                             A.Rotate(limit=30,p=1.0,border_mode=cv2.BORDER_CONSTANT),A.HorizontalFlip(p=0.5),
                             A.VerticalFlip(p=0.1),A.Normalize((0),(1)),
                             ToTensorV2()])

val_transform = A.Compose([A.Resize(height=256,width=256),
                           A.Normalize((0),(1)),
                           ToTensorV2()])

trainset = DATASET("/content/drive/MyDrive/Segmentation/MRI SEG/MRI/Train Images","/content/drive/MyDrive/Segmentation/MRI SEG/MRI/Train Masks",transform=train_transform)
valset = DATASET("/content/drive/MyDrive/Segmentation/MRI SEG/MRI/Train Images","/content/drive/MyDrive/Segmentation/MRI SEG/MRI/Train Masks",transform = val_transform)

n_train = len(trainset)
indices = list(range(n_train))
np.random.shuffle(indices)
split=int(np.floor(0.05*n_train))
train_idx,val_idx = indices[split:],indices[:split]
tsampler,vsampler = SubsetRandomSampler(train_idx),SubsetRandomSampler(val_idx)
train_loader = DataLoader(trainset,sampler=tsampler,batch_size = 4)
valid_loader = DataLoader(valset,sampler=vsampler,batch_size = 4)

model = UNET()
# model.load_state_dict(torch.load("/content/drive/MyDrive/Segmentation/MRI SEG/Model/unet.pth"))
model=model.cuda()
optimiser = optim.Adam(model.parameters(),lr=0.0001)
scaler = torch.cuda.amp.GradScaler()
criterion = nn.BCEWithLogitsLoss()
min_acc = 0
epochs = 30
num_correct = 0
num_pixels = 0
dice_score = 0
v_loss = 0
for batch_idx, (images, labels) in enumerate(valid_loader):
    # move tensors to GPU if CUDA is available
    labels = labels.unsqueeze(1)
    images, labels = images.cuda(), labels.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    with torch.cuda.amp.autocast():
        output = model(images)
        loss=criterion(output,labels)
        output = torch.sigmoid(output)
        output = (output>0.5).float()
        loss=criterion(output,labels)
        v_loss += loss.cpu().item()
    num_correct += (output == labels).sum()
    num_pixels += torch.numel(output)
validation_losses.append(v_loss/len(valid_loader))
dice_score += (2*(output*labels).sum().cpu().item())/((output+labels).cpu().sum().item() + 1e-8)
dice_score = dice_score/len(valid_loader)
dice_scores.append(dice_score)
print("\nEPOCH:",0,"\tDice Score:",dice_score)
# calculate the batch loss

for e in range(1,epochs+1):
    model.train()
    loop = tqdm(train_loader)
    for images,labels in loop:
        labels = labels.unsqueeze(1)
        images,labels=images.cuda(),labels.cuda()
        optimiser.zero_grad()
        with torch.cuda.amp.autocast():
                    output=model.forward(images)
                    loss=criterion(output,labels)
        scaler.scale(loss).backward()
        scaler.step(optimiser)
        scaler.update()
        loop.set_postfix(loss=loss.item())
        training_losses.append(loss.cpu().item())
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()
    # iterate over test data
    v_loss = 0
    for batch_idx, (images, labels) in enumerate(valid_loader):
    # move tensors to GPU if CUDA is available
        labels = labels.unsqueeze(1)
        images, labels = images.cuda(), labels.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
        with torch.cuda.amp.autocast():
            output = model(images)
            loss=criterion(output,labels)
            output = torch.sigmoid(output)
            output = (output>0.5).float()
            v_loss += loss.cpu().item()
        
        num_correct += (output == labels).sum()
        num_pixels += torch.numel(output)
        dice_score += (2*(output*labels).sum().cpu().item())/((output+labels).cpu().sum().item() + 1e-8)
    validation_losses.append(v_loss/len(valid_loader))
    dice_score = dice_score/len(valid_loader)
    dice_scores.append(dice_score)
    print("\nEPOCH:",e,"\tDice Score:",dice_score,"\tValidation Loss:",v_loss/len(valid_loader))
    # calculate the batch loss
        
    if dice_score>min_acc:
        print("Saving model")
        min_acc=dice_score
        torch.save(model.state_dict(),"/content/drive/MyDrive/Segmentation/MRI SEG/Model/unet_2.pth")

plotVals(
    losses=training_losses,
    ylim=(0, max(training_losses)+0.5),
    f_name="Training Loss",
    x_label="Training Step",
    y_label="Training Loss",
)
plotVals(
    losses=validation_losses,
    ylim=(0, max(validation_losses)+0.5),
    f_name="Validation Loss",
    x_label="Epoch",
    y_label="Validation Loss",
)
plotVals(
    losses=dice_scores,
    ylim=(0, 1.0),
    f_name="Dice Score",
    x_label="Epoch",
    y_label="Dice Score",
)


 29%|██▉       | 119/407 [00:19<00:48,  5.99it/s, loss=0.0062]

torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])


 30%|██▉       | 121/407 [00:19<00:47,  6.08it/s, loss=0.00672]

torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])


 30%|███       | 123/407 [00:20<00:45,  6.19it/s, loss=0.00927]

...

torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])


100%|█████████▉| 405/407 [01:06<00:00,  6.14it/s, loss=0.0033]

torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])


100%|██████████| 407/407 [01:06<00:00,  6.09it/s, loss=0.00598]


torch.Size([1, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([1, 3, 256, 256])

EPOCH: 30 	Dice Score: 0.9592985716101889 	Validation Loss: 0.0051001524341038685
